In [293]:
import pandas as pd
import numpy as np
import sklearn as sk
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.linear_model import Ridge
from sklearn.grid_search import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.cluster import AgglomerativeClustering

In [284]:
df = pd.read_csv('data/preprocessed_resume.csv', index_col=0)
#df = df.iloc[:20]
df.shape

(2039, 14)

Хотел впихнуть N-граммы, комп не вывез:)

In [285]:
vectorizer = TfidfVectorizer(ngram_range=(1, 1))
bag = vectorizer.fit_transform(df.text)
bag = bag.toarray()
bag.shape

(2039, 35442)

In [286]:
x = df.drop(['salary', 'text'], axis=1).values.copy()
y = df.salary.values
y.shape

(2039,)

**2 подход**

находим наиболее важные фичи в tf-idf

In [295]:
chooser = SelectKBest(chi2, k=(10000))
best_features = chooser.fit_transform(bag, y)
best_features.shape

(2039, 10000)

Хи-квадрат, она тут неспроста, надо разобраться, как именно отбираем фичи.

In [296]:
X_smart = np.concatenate((x, best_features), axis=1)
X_train_smart, X_test_smart, y_train_smart, y_test_smart = train_test_split(
    X_smart, y, test_size=0.1)
X_train_smart.shape

(1835, 10012)

Почему так странно? 

Задача регрессии, то есть очень чувствительна к размеру листьев. Можно попробовать объяснить интуицию, но вообще есть такая эвристика - в задачах регресси в рандом форесте брать половину или треть признаков. 

Важное замечание!
Импьюрити фич очень неравномерны, поэтому если в выборку не попадают, к примеру, первые 12 фич, то это поддерево ломает всю модель. Поэтому берем много фич.

In [305]:
regressor_sm = RandomForestRegressor(max_depth=20, max_features=5000, n_estimators=1000)
regressor_sm.fit(X_train_smart, y_train_smart)
regressor_sm.score(X_test_smart, y_test_smart)

0.6026124824816339

Несколько регрессоров в рандом форесте - аналог кросс-валидации. Увеличивая их число повышаем обобщающую способность модели. 

In [310]:
y_pred = regressor_sm.predict(X_test_smart)
r = np.linalg.norm((y_pred - y_test_smart)) / np.sqrt(y_test_smart.shape[0])
r

24933.203335335438

Среднеквадратичная ошибка. Пойдет...

In [315]:
y_train_smart.mean()

65045.51825613079

In [307]:
np.bincount((np.abs(y_pred - y_test_smart) < 10000))

array([123,  81])

Доля предиктов, в которых ошиблись меньше, чем на 10 к.

In [317]:
np.bincount((np.abs(y_pred - y_test_smart) < 10000))[1] / y_test_smart.shape[0]

0.39705882352941174